## Importing Modules To Connect Youtube API 

#### Importing all the modules which are required to pull data from Youtube API which are as follows -:
* Google APi Client
* Pandas
* Json
* CSV


In [15]:
from googleapiclient.discovery import build
import pandas as pd
import json
import csv

## Requesting Data From Youtube Api. 
* Response variable has all the `raw` data.
* All channels related stats are stored in data frame
* We have to find playlist ids for all the channels that is why loop is used to iterate over rows.
* Stored in a variable named playlist_ids
* Finally, return playlist_ids

In [16]:
def get_playlist_ids(youtube,channel_id):
    #requesting data from youtube api
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id = ','.join(channel_id),
    )
    #executing response
    response =request.execute()

    print(response)

    # storing data in all_data list

    all_data = []

    #all important information about the channel

    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'], 
                'views': item['statistics']['viewCount'], 
                'totalViews': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']

        }

    #appending data into list
    all_data.append(data)

    #converting data into a dataframe

    data_frame = pd.DataFrame(all_data)


    # making playlist_id for all the channels

    playlist_id = []

    # extracting playlist_ids of all the channels

    for index, rows in data_frame.iterrows():
        if rows['playlistId'] not in playlist_id: 
            playlist_id.append(rows['playlistId'])


    # return playlist_id


    return playlist_id





## Extracting Video Id's From Playlist Id's. 
* Response variable has all the `raw` data.
* Cleaning data and returning video id as a list
* Next page token is to get all the videos. 

In [17]:
def get_video_ids(youtube, playlist_id):
    video_ids = []

    #requesting data from youtube api

    request =youtube.playlistItems().list(
        part = "snippet, contentDetails", 
        playlistId = ','.join(playlist_id),
    )
    #executing response

    response = request.execute()

    #appending video ids in a list


    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])



    #parsing to next pages with nextpagetoken


    next_page_token = response.get('nextPageToken')

    while next_page_token is not None:
        #requesting data from youtube api

        request = youtube.playlistItems().list(
            part = "snippet, contentDetails", 
            playlistId = ','.join(playlist_id),
            maxResults = 50,
            pageToken = next_page_token
            )
        
        response = request.execute()
        #executing response
        
        next_page_token = response.get('nextPageToken')

        #appending video ids in a list
        
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

    return video_ids

In [18]:
def get_video_stats(youtube, video_ids):
    all_video_info = []


    for ids in video_ids:
        request = youtube.videos().list(
            part = "snippet,contentDetails,statistics",
            id = ids
        )
        response = request.execute()

        for video in response["items"]:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                        'statistics':['viewCount','likeCount','commentCount'],
                        'contentDetails':['duration']

            }
            video_info = {}
            video_info['video_id'] = video['id']

            for keys in stats_to_keep.keys():
              for values in stats_to_keep[keys]:
                  try: 
                      video_info[values] = video[keys][values]
                  except: 
                      video_info[values] = None
            
            
            all_video_info.append(video_info)

    data_frame = pd.DataFrame(all_video_info)



    return data_frame


In [19]:
# def get_comments_in_videos(youtube, video_ids):
#     """
#     Get top level comments as text from all videos with given IDs (only the first 10 comments due to quote limit of Youtube API)
#     Params:
    
#     youtube: the build object from googleapiclient.discovery
#     video_ids: list of video IDs
    
#     Returns:
#     Dataframe with video IDs and associated top level comment in text.
    
#     """
#     all_comments = []
    
#     for video_id in video_ids:
#         try:   
#             request = youtube.commentThreads().list(
#                 part="snippet,replies",
#                 videoId=video_id
#             )
#             response = request.execute()
        
#             comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]]
#             comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

#             all_comments.append(comments_in_video_info)
            
#         except: 
#             # When error occurs - most likely because comments are disabled on a video
#             print('Could not get comments for video ' + video_id)

#     data_frame = pd.DataFrame(all_comments)

#     data_frame.to_csv("comments_data")
        
#     return data_frame
     

In [ ]:
#Driver Code
#get credentials and create api client
api_service_name = "youtube"
api_version = "v3"

#Youtube Api Key

api_key = "Your API Key Here"


#Building Youtube to call the API using API Key
youtube = build(
        api_service_name,api_version,developerKey=api_key
    )


# ALL Channell id that we have to extract data from

channel_id = ["UCcGcC0DCDXyqSPdpskY99CQ", 
                #more channels here
    ]

# getting channel stats

playlist_ids = get_playlist_ids(youtube,channel_id)

# getting video ids for all the videos

video_ids = get_video_ids(youtube,playlist_ids)


video_stats = get_video_stats(youtube,video_ids)

# comments_in_videos = get_comments_in_videos(youtube,video_ids)




In [24]:
# importing isodate to make duration visible
import isodate
import csv 
from datetime import datetime

now = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
#Cleaning duration date
video_stats['durationSecs'] = video_stats['duration'].apply(lambda x: isodate.parse_duration(x))

#converting duration to seconds

video_stats['durationSecs'] = video_stats['durationSecs'].astype('timedelta64[s]')
#counitng tagcount
video_stats['tagCount'] = video_stats['tags'].apply(lambda x:0 if x is None else len(x))


video_stats
video_stats_csv = f"video_stats{now}.csv"
video_stats.to_csv(video_stats_csv, index = False)

## Importing gspread module, pandas & service account. 

### Modules and its use cases.

* Gspread: To read, write & update data into google sheets. 
* pprint: To present the data in a better format. 
* service account: To connect with `google api`. 


In [25]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import csv

scope = ["https://spreadsheets.google.com/feeds",
        'https://www.googleapis.com/auth/spreadsheets',
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)

google_sheet = gspread.authorize(creds)

### Importing Data into CSV

* Setting the permissions
* Changing worksheet name
* Importing CSV data to google sheets

In [26]:

#Selecting sheet by ID & Setting The Permissions
google_sheet_id = 'Your Google Sheets Key here'
google_sheet.insert_permission(
    google_sheet_id,
    None,
    perm_type='anyone',
    role='writer'
)

#Writing Data into from csv to Master-worksheet

spread_sheet = google_sheet.open_by_key(google_sheet_id)

# spread_sheet.values_update(
#     'payments_ques_3',
#     params={'valueInputOption': 'USER_ENTERED'},
#     body={'values': list(csv.reader(open(sheet_csv_3)))}
#     )

print("data exported")

spread_sheet.values_update(
    'raw_data',
    params={'valueInputOption': 'USER_ENTERED'},
    body={'values': list(csv.reader(open(video_stats_csv)))}
    )

data exported


{'spreadsheetId': '1vng4WhXtTr48yLS5wz0RmDT_lNydZFNc9Z-ePBtqzM0',
 'updatedRange': 'raw_data!A1:L258',
 'updatedRows': 258,
 'updatedColumns': 12,
 'updatedCells': 3096}

In [54]:
video_stats

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,durationSecs,tagCount
0,I3xRvxf6zk4,Deepak Tulsyan,What’s my Name 🔥😍 #deepaktulsyan,"Hit Like, if you like this video.\nDon't forge...",None,NaN,40128.0,1595.0,23.0,PT15S,15.0,0
1,GyPSjEItdNM,Deepak Tulsyan,Tabahi 🔥 | Badshah | Deepak Tulsyan | G M Danc...,Click Here to Subscribe:- http://bit.ly/Deepak...,None,NaN,73771.0,2512.0,31.0,PT22S,22.0,0
2,BLqBdiTvXn0,Deepak Tulsyan,Doobey 💙| Deepak Tulsyan Choreography #josh,Follow me on Josh app for more exclusive conte...,None,NaN,191477.0,9617.0,57.0,PT37S,37.0,0
3,1zE7TyF7Gl8,Deepak Tulsyan,Banke Titali 😍🔥 Dancing with @Akshita Goel #j...,Follow me on Josh app for more exclusive conte...,None,NaN,138299.0,6599.0,60.0,PT25S,25.0,0
4,SznbFm6jfXU,Deepak Tulsyan,Easy Tutorial for this Trend 💙💜 #josh #trend,Follow me on Josh app for more exclusive conte...,None,NaN,722996.0,36804.0,84.0,PT29S,29.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
252,ArPozq_q_xs,Deepak Tulsyan,How to Dance | Tutorial no. 3 | Sun Saathiya |...,Steps for Complete Beginners! Deepak welcomes ...,"[ABCD 2, The Walt Disney Company (Production C...",NaN,112964.0,1131.0,NaN,PT4M9S,249.0,13
253,_FOm1FMkccg,Deepak Tulsyan,How to dance | Tutorial no. 2 | Basic footwork,This is a 3 steps tutorial video. Watch it cou...,"[Jazz (Musical Genre), Ballet Dancer (Professi...",NaN,83638.0,1303.0,NaN,PT1M17S,77.0,6
254,plbfhxhYIsY,Deepak Tulsyan,Sooraj Duba Hai | Roy | Freestyle Dance Choreo...,For more Videos....\nhttps://www.facebook.com/...,[Dance (Interest)],NaN,10004.0,160.0,NaN,PT38S,38.0,1
255,LsT6BfBZYCw,Deepak Tulsyan,Must Watch | Abhi toh party Suru hui hai | Fre...,For more Videos....\nhttps://www.facebook.com/...,"[Dance (Interest), Kid Cudi (Music Video Perfo...",NaN,10830.0,298.0,NaN,PT1M20S,80.0,5
